## Work
1. 請比較使用不同層數以及不同 Dropout rate 對訓練的效果
2. 將 optimizer 改成使用 Adam 並加上適當的 dropout rate 檢視結果

In [1]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [30]:
from keras.layers import Dropout

def build_mlp(input_shape, output_units = 10, num_neurons=[512,256,128], dropout_rate=0.2):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i+1))(x)
            
        x = Dropout(dropout_rate)(x)    
        
    out = keras.layers.Dense(units=output_units, activation='softmax', name='ouput')(x)
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
            
    return model

In [31]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
Dropout_RATE = [0.2,0.3]

In [32]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for Dropout_EXP in Dropout_RATE:
    model = build_mlp(input_shape=x_train.shape[1:], dropout_rate=Dropout_EXP)
    model.summary()
    optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    exp_name_tag = "exp-dropout-%s" % str(Dropout_EXP)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}    


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0   

50000/50000 [==============================] - 8s 156us/step - loss: 1.4092 - accuracy: 0.4940 - val_loss: 1.4028 - val_accuracy: 0.5002
Epoch 47/50
50000/50000 [==============================] - 8s 156us/step - loss: 1.4015 - accuracy: 0.4950 - val_loss: 1.3895 - val_accuracy: 0.5030
Epoch 48/50
50000/50000 [==============================] - 8s 156us/step - loss: 1.4074 - accuracy: 0.4917 - val_loss: 1.4079 - val_accuracy: 0.4970
Epoch 49/50
50000/50000 [==============================] - 8s 157us/step - loss: 1.3990 - accuracy: 0.4968 - val_loss: 1.3924 - val_accuracy: 0.5016
Epoch 50/50
50000/50000 [==============================] - 8s 157us/step - loss: 1.3978 - accuracy: 0.4971 - val_loss: 1.3963 - val_accuracy: 0.4939
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 3072)              0         
____________________________________________________

50000/50000 [==============================] - 7s 149us/step - loss: 1.6048 - accuracy: 0.4219 - val_loss: 1.5719 - val_accuracy: 0.4464
Epoch 42/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5973 - accuracy: 0.4233 - val_loss: 1.5336 - val_accuracy: 0.4560
Epoch 43/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5924 - accuracy: 0.4264 - val_loss: 1.5512 - val_accuracy: 0.4529
Epoch 44/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5895 - accuracy: 0.4255 - val_loss: 1.5556 - val_accuracy: 0.4477
Epoch 45/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5958 - accuracy: 0.4253 - val_loss: 1.5644 - val_accuracy: 0.4560
Epoch 46/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5846 - accuracy: 0.4275 - val_loss: 1.5293 - val_accuracy: 0.4628
Epoch 47/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5846 - accuracy: 0.4292 - val_loss:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()